## Load data

In [1]:
import pandas as pd

In [2]:
def process_data_for_model(data_path):
    df = pd.read_csv(data_path)

    df = df[['time', 'close', 'volumeto']].copy()
    df = df.rename(columns={'volumeto': 'volume'})
    df['id'] = 'BTC'
    df['time'] = pd.to_datetime(df['time']).dt.tz_localize(None)
    df['return'] = df['close'].pct_change().shift(-1)  # 1-step-ahead return
    df=df.dropna(subset=['return'])
    df.info()
    df.head()

    return df

## AutoMLStrategy

In [8]:
import sys
import numpy as np
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

sys.path.append("../src")

from strategy import Strategy
from backtest import run_backtest

class AutoMLStrategy(Strategy):
    def __init__(self, initial_capital=10000, df=None, lookback=1000):
        super().__init__(initial_capital)
        self.has_bought = False
        self.lookback = lookback           # how many bars to keep
        self.window   = pd.DataFrame()  # seed with training tail
        self.prev_close = None


        train_df = TimeSeriesDataFrame.from_data_frame(
            df.head(10),          # drop last NaN caused by shift
            timestamp_column='time',
            id_column='id',
            static_features_df=None
        )
        self.model = TimeSeriesPredictor(
            target='return',
            prediction_length=1,
            eval_metric='rmse',
            quantile_levels=[0.05, 0.25, 0.5, 0.75, 0.95],  # gives you a distribution
            freq='1H'             # or '1H', '5min', …  match your data
            # presets='medium_quality'
        )
        

        self.model.fit(train_df)
        self.model.save()

    # -----------------------------------------------------------------
    # called once per new bar by the back-testing engine
    # -----------------------------------------------------------------
    def process_bar(self, bar):
        # ── 1 ▪ add id and compute return ───────────────────────────
        bar['id'] = 'BTC'
        if self.prev_close is None:
            bar['return'] = np.nan                # first bar
        else:
            bar['return'] = (bar['close'] - self.prev_close) / self.prev_close
        self.prev_close = bar['close']

        # ── 2 ▪ update rolling window ───────────────────────────────
        self.window = (
            pd.concat([self.window, pd.DataFrame([bar])])
              .tail(self.lookback)
              .reset_index(drop=True)
        )
        # self.window.head()

        # Need at least one non-NaN return to forecast
        if self.window['return'].count() < 2:
            return

        # ── 3 ▪ TimeSeriesDataFrame for AG ──────────────────────────
        latest_tsd = TimeSeriesDataFrame.from_data_frame(
            self.window,
            timestamp_column='time',
            id_column='id',
        )
        # latest_tsd.head()
        # print(latest_tsd)

        # ── 4 ▪ Forecast next-bar return distribution ───────────────
        self.fcst = self.model.predict(latest_tsd).iloc[0]


    # -----------------------------------------------------------------
    # turn forecast + risk estimate into trading instruction
    # -----------------------------------------------------------------
    def get_signal(self) -> str:
        """
        BUY  :  μ > k·σ  and entire 90 % interval is positive
        SELL :  μ < −k·σ and entire 90 % interval is negative
        HOLD :  otherwise
        """

        # Need enough history to compute σ -- otherwise stay flat
        if len(self.window) < self.lookback:
            return "hold"

        sigma = self.window["return"].std(ddof=1)     # sample std
        k_sigma = self.thresh_factor * sigma

        mu   = self.fcst["0.5"]   # median / mean
        q05  = self.fcst["0.05"]
        q95  = self.fcst["0.95"]

        print(mu, k_sigma, q05, q95)
        call = 'hold'

        # --- decision logic ------------------------------------------
        if (mu > k_sigma) and (q05 > 0) and not self.has_position:
            self.has_position = True
            call = "buy"

        if (mu < -k_sigma) and (q95 < 0) and self.has_position:
            self.has_position = False
            call = "sell"

        print(call)
        return call

In [9]:
df = process_data_for_model('../data/btc_hour.csv')
training_df = df.copy().head(20)
test_df = df.copy().tail(1000)

strategy = AutoMLStrategy(initial_capital=10000, df=training_df, lookback=1000)

training_df.head()

/Users/jiro/play/nstrade/.venv/lib/python3.12/site-packages/autogluon/timeseries/predictor.py:198: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  offset = pd.tseries.frequencies.to_offset(self.freq)
Frequency '1H' stored as 'h'
Beginning AutoGluon training...
AutoGluon will save models to '/Users/jiro/play/nstrade/notebooks/AutogluonModels/ag-20250517_112123'


<class 'pandas.core.frame.DataFrame'>
Index: 118058 entries, 0 to 118057
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   time    118058 non-null  datetime64[ns]
 1   close   118058 non-null  float64       
 2   volume  118058 non-null  float64       
 3   id      118058 non-null  object        
 4   return  118058 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 5.4+ MB


=================== System Info ===================
AutoGluon Version:  1.3.0
Python Version:     3.12.8
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 23.6.0: Thu Mar  6 22:08:50 PST 2025; root:xnu-10063.141.1.704.6~1/RELEASE_ARM64_T8112
CPU Count:          8
GPU Count:          0
Memory Avail:       4.84 GB / 16.00 GB (30.2%)
Disk Space Avail:   21.65 GB / 228.27 GB (9.5%)

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': RMSE,
 'freq': 'h',
 'hyperparameters': 'default',
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_length': 1,
 'quantile_levels': [0.05, 0.25, 0.5, 0.75, 0.95],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'return',
 'verbosity': 4}

Provided train_data has 10 rows, 1 time series. Median time series length is 10 (min=10, max=10). 

Provided data contains following columns:
	target: 'return'
	past_covariates:
		cat

,time,close,volume,id,return
0,2011-11-28 00:00:00,2.497,8571.74,BTC,0.000400
1,2011-11-28 01:00:00,2.498,1698.15,BTC,-0.004404
2,2011-11-28 02:00:00,2.487,1559.13,BTC,-0.018094
3,2011-11-28 03:00:00,2.442,6519.05,BTC,0.008190
4,2011-11-28 04:00:00,2.462,10667.19,BTC,0.000812


In [96]:
strategy.window.head()

,time,close,volume,id,return
0,2024-03-26 12:00:00,71113.61,8.020626e+07,BTC,0.005855
1,2024-03-26 13:00:00,70402.66,1.554150e+08,BTC,-0.009997
2,2024-03-26 14:00:00,70321.85,3.032331e+08,BTC,-0.001148
3,2024-03-26 15:00:00,70270.72,1.173906e+08,BTC,-0.000727
4,2024-03-26 16:00:00,69758.08,1.291990e+08,BTC,-0.007295


In [99]:
print(strategy.window.dtypes)
print(strategy.window.head())

time      datetime64[ns]
close            float64
volume           float64
id                object
return           float64
dtype: object
                 time     close        volume   id    return
0 2024-03-26 13:00:00  70402.66  1.554150e+08  BTC -0.009997
1 2024-03-26 14:00:00  70321.85  3.032331e+08  BTC -0.001148
2 2024-03-26 15:00:00  70270.72  1.173906e+08  BTC -0.000727
3 2024-03-26 16:00:00  69758.08  1.291990e+08  BTC -0.007295
4 2024-03-26 17:00:00  70078.50  1.108090e+08  BTC  0.004593


In [ ]:

autogluon_res = run_backtest(
    lambda initial_capital: strategy,
    test_df,
    initial_capital=10000
)


Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble
Prediction order: ['RecursiveTabular', 'DynamicOptimizedTheta', 'AutoETS', 'SeasonalNaive', 'WeightedEnsemble']
Shortening all series to at most 1000002
Shortening all time series to at most 2500


<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 117058 to 118057
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   time    1000 non-null   datetime64[ns]
 1   close   1000 non-null   float64       
 2   volume  1000 non-null   float64       
 3   id      1000 non-null   object        
 4   return  1000 non-null   float64       
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 46.9+ KB
hold
hold


Shortening all time series to at most 2500
Shortening all time series to at most 2500
Cached predictions saved to /Users/jiro/play/nstrade/notebooks/AutogluonModels/ag-20250517_112123/models/cached_predictions.pkl
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble
Prediction order: ['RecursiveTabular', 'DynamicOptimizedTheta', 'AutoETS', 'SeasonalNaive', 'WeightedEnsemble']
Shortening all series to at most 1000002
Shortening all time series to at most 2500
Shortening all time series to at most 2500
Shortening all time series to at most 2500
Cached predictions saved to /Users/jiro/play/nstrade/notebooks/AutogluonModels/ag-20250517_112123/models/cached_predictions.pkl
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble
Prediction order: ['RecursiveTabular', 'DynamicOptimizedTheta', 'AutoETS', 'SeasonalNaive', 'WeightedEnsemble']
Shortening all series to at most 1000002
Shortenin

hold
hold
hold


Cached predictions saved to /Users/jiro/play/nstrade/notebooks/AutogluonModels/ag-20250517_112123/models/cached_predictions.pkl
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble
Prediction order: ['RecursiveTabular', 'DynamicOptimizedTheta', 'AutoETS', 'SeasonalNaive', 'WeightedEnsemble']
Shortening all series to at most 1000002
Shortening all time series to at most 2500
Shortening all time series to at most 2500
Shortening all time series to at most 2500
Cached predictions saved to /Users/jiro/play/nstrade/notebooks/AutogluonModels/ag-20250517_112123/models/cached_predictions.pkl
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble
Prediction order: ['RecursiveTabular', 'DynamicOptimizedTheta', 'AutoETS', 'SeasonalNaive', 'WeightedEnsemble']
Shortening all series to at most 1000002
Shortening all time series to at most 2500
Shortening all time series to at most 2500
Shortenin

...

Cached predictions saved to /Users/jiro/play/nstrade/notebooks/AutogluonModels/ag-20250517_112123/models/cached_predictions.pkl
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble


hold


Prediction order: ['RecursiveTabular', 'DynamicOptimizedTheta', 'AutoETS', 'SeasonalNaive', 'WeightedEnsemble']
Shortening all series to at most 1000002
Shortening all time series to at most 2500
Shortening all time series to at most 2500
Shortening all time series to at most 2500
Cached predictions saved to /Users/jiro/play/nstrade/notebooks/AutogluonModels/ag-20250517_112123/models/cached_predictions.pkl
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble


hold


Prediction order: ['RecursiveTabular', 'DynamicOptimizedTheta', 'AutoETS', 'SeasonalNaive', 'WeightedEnsemble']
Shortening all series to at most 1000002
Shortening all time series to at most 2500
Shortening all time series to at most 2500
Shortening all time series to at most 2500


## Development // Drafts // WIP

In [ ]:
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

df = df.head(1000)

train_df = TimeSeriesDataFrame.from_data_frame(
    df,          # drop last NaN caused by shift
    timestamp_column='timestamp',
    id_column='id',
    static_features_df=None
)


model = TimeSeriesPredictor(
    target='return',
    prediction_length=1,
    eval_metric='rmse',
    verbosity=4
)


In [2]:
run_backtest(AutoMLStrategy, df)

NameError: name 'run_backtest' is not defined

In [ ]:
# create a new strategy
class AutoMLStrategy(Strategy):
    def __init__(self, initial_capital=10000):
        super().__init__(initial_capital)
        self.has_bought = False
        self.model = model

    def process_bar(self, bar):
        self.current_bar = bar
        self.model.predict(bar)

    def get_signal(self):
        if not self.has_bought:
            self.has_bought = True